In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.ensemble import IsolationForest
import seaborn as sns
from typing import Tuple, Dict, List
import community  # python-louvain package
from collections import Counter
import warnings
from tabulate import tabulate
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

# Suppress specific FutureWarnings from Seaborn
warnings.filterwarnings("ignore", category=FutureWarning, module='seaborn')

def load_graph(nodes_file: str, edges_file: str) -> Tuple[nx.Graph, pd.DataFrame, pd.DataFrame]:
    """
    Load network data from CSV files and create a NetworkX graph.

    Parameters:
    - nodes_file: Path to the nodes CSV file.
    - edges_file: Path to the edges CSV file.

    Returns:
    - G: NetworkX graph.
    - nodes_df: DataFrame containing node attributes.
    - edges_df: DataFrame containing edge attributes.
    """
    nodes_df = pd.read_csv(nodes_file)
    edges_df = pd.read_csv(edges_file)
    G = nx.Graph()
    
    # Add node attributes as a dictionary
    node_attrs = nodes_df.set_index('Id').to_dict('index')
    G.add_nodes_from([(node, attrs) for node, attrs in node_attrs.items()])
    
    # Add edges with weights if available
    if 'Weight' in edges_df.columns:
        G.add_weighted_edges_from([(row['Source'], row['Target'], row['Weight']) 
                                 for _, row in edges_df.iterrows()])
    else:
        G.add_edges_from([(row['Source'], row['Target']) 
                         for _, row in edges_df.iterrows()])
    
    return G, nodes_df, edges_df

def calculate_network_metrics(G: nx.Graph) -> Dict:
    """
    Calculate comprehensive network metrics, including spectral radius and Fiedler eigenvalue.

    Parameters:
    - G: NetworkX graph.

    Returns:
    - metrics: Dictionary containing network metrics.
    """
    metrics = {
        'Average Clustering': nx.average_clustering(G),
        'Network Density': nx.density(G),
        'Average Degree': sum(dict(G.degree()).values()) / G.number_of_nodes(),
        'Components': nx.number_connected_components(G)
    }
    
    # Check if the graph is connected
    if nx.is_connected(G):
        # Average Path Length and Diameter
        metrics['Average Path Length'] = nx.average_shortest_path_length(G)
        metrics['Network Diameter'] = nx.diameter(G)
        
        # Spectral Radius
        A = nx.adjacency_matrix(G).todense()
        eigvals = np.linalg.eigvals(A)
        spectral_radius = np.max(np.abs(eigvals))
        metrics['Spectral Radius'] = spectral_radius
        
        # Fiedler Eigenvalue
        L = nx.laplacian_matrix(G).todense()
        eigvals_L = np.linalg.eigvalsh(L)  # Since Laplacian is symmetric
        if len(eigvals_L) >= 2:
            fiedler_eigval = eigvals_L[1]
            metrics['Fiedler Eigenvalue'] = fiedler_eigval
        else:
            metrics['Fiedler Eigenvalue'] = np.nan
    else:
        metrics['Average Path Length'] = np.nan
        metrics['Network Diameter'] = np.nan
        metrics['Spectral Radius'] = np.nan
        metrics['Fiedler Eigenvalue'] = np.nan
    
    try:
        communities = community.best_partition(G)
        metrics['Number of Communities'] = len(set(communities.values()))
    except Exception as e:
        print(f"Community detection failed: {e}")
        metrics['Number of Communities'] = np.nan
    
    return metrics

def calculate_node_metrics(G: nx.Graph) -> pd.DataFrame:
    """
    Calculate comprehensive node-level metrics.

    Parameters:
    - G: NetworkX graph.

    Returns:
    - node_metrics_df: DataFrame containing node-level metrics.
    """
    metrics = {
        'Degree Centrality': nx.degree_centrality(G),
        'Betweenness Centrality': nx.betweenness_centrality(G),
        'Closeness Centrality': nx.closeness_centrality(G),
        'Eigenvector Centrality': nx.eigenvector_centrality(G, max_iter=1000),
        'Clustering Coefficient': nx.clustering(G),
        'PageRank': nx.pagerank(G)
    }
    node_metrics_df = pd.DataFrame(metrics).sort_index()
    
    # Compute Fiedler Vector if the graph is connected
    if nx.is_connected(G):
        L = nx.laplacian_matrix(G).todense()
        eigvals_L, eigvecs_L = np.linalg.eig(L)
        idx = eigvals_L.argsort()
        eigvals_L = eigvals_L[idx]
        eigvecs_L = eigvecs_L[:, idx]
        
        if len(eigvals_L) >= 2:
            fiedler_vector = eigvecs_L[:, 1]
            node_metrics_df['Fiedler Vector'] = fiedler_vector
        else:
            node_metrics_df['Fiedler Vector'] = np.nan
    else:
        node_metrics_df['Fiedler Vector'] = np.nan
    
    return node_metrics_df

def analyze_changes(G1: nx.Graph, G2: nx.Graph) -> Dict:
    """
    Analyze structural changes between two network snapshots.

    Parameters:
    - G1: First NetworkX graph.
    - G2: Second NetworkX graph.

    Returns:
    - changes: Dictionary containing structural changes.
    """
    changes = {
        'New Nodes': len(set(G2.nodes()) - set(G1.nodes())),
        'Removed Nodes': len(set(G1.nodes()) - set(G2.nodes())),
        'New Edges': len(set(G2.edges()) - set(G1.edges())),
        'Removed Edges': len(set(G1.edges()) - set(G2.edges())),
    }
    
    degrees1 = Counter(dict(G1.degree()).values())
    degrees2 = Counter(dict(G2.degree()).values())
    changes['Degree Distribution Change'] = sum((degrees2 - degrees1).values())
    
    return changes

def calculate_k_core(G: nx.Graph) -> Dict:
    """Calculate core numbers for the graph."""
    return nx.core_number(G)

def calculate_edge_betweenness(G: nx.Graph) -> Dict[Tuple, float]:
    """Calculate edge betweenness centrality."""
    return nx.edge_betweenness_centrality(G)

def calculate_node_strength(G: nx.Graph) -> Dict:
    """Calculate node strength."""
    node_strength = {}
    for u, v, data in G.edges(data=True):
        weight = data.get('Weight', 1)
        node_strength[u] = node_strength.get(u, 0) + weight
        node_strength[v] = node_strength.get(v, 0) + weight
    return node_strength

def calculate_assortativity(G: nx.Graph) -> float:
    """Calculate degree assortativity coefficient."""
    return nx.degree_assortativity_coefficient(G)

def calculate_local_efficiency(G: nx.Graph) -> float:
    """Calculate average local efficiency."""
    return nx.local_efficiency(G)

def assess_network_robustness(G: nx.Graph, removal_fraction=0.1) -> Dict:
    """
    Assess network robustness by simulating targeted attacks.
    
    Parameters:
    - G: NetworkX graph
    - removal_fraction: Fraction of nodes/edges to remove (default 0.1 or 10%)
    
    Returns:
    - Dictionary containing the size of largest connected component after node/edge removal
    """
    results = {}
    original_largest_component_size = len(max(nx.connected_components(G), key=len))
    
    # Simulate node removal attack
    edge_betweenness = nx.edge_betweenness_centrality(G)
    nodes_sorted = sorted(G.degree, key=lambda x: x[1], reverse=True)
    nodes_to_remove = int(removal_fraction * G.number_of_nodes())
    nodes_removed = [node for node, degree in nodes_sorted[:nodes_to_remove]]
    
    G_temp = G.copy()
    G_temp.remove_nodes_from(nodes_removed)
    if G_temp.number_of_nodes() > 0:
        largest_connected_component = max(nx.connected_components(G_temp), key=len)
        results['Largest_Connected_Component_After_Node_Removal'] = len(largest_connected_component)
        results['Connected_Component_Size_Change_After_Node_Removal'] = (
            len(largest_connected_component) - original_largest_component_size
        )
    else:
        results['Largest_Connected_Component_After_Node_Removal'] = 0
        results['Connected_Component_Size_Change_After_Node_Removal'] = -original_largest_component_size
    
    # Simulate edge removal attack
    edges_sorted = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)
    edges_to_remove = int(removal_fraction * G.number_of_edges())
    edges_removed = [edge for edge, centrality in edges_sorted[:edges_to_remove]]
    
    G_temp = G.copy()
    G_temp.remove_edges_from(edges_removed)
    if G_temp.number_of_edges() > 0:
        largest_connected_component = max(nx.connected_components(G_temp), key=len)
        results['Largest_Connected_Component_After_Edge_Removal'] = len(largest_connected_component)
        results['Connected_Component_Size_Change_After_Edge_Removal'] = (
            len(largest_connected_component) - original_largest_component_size
        )
    else:
        results['Largest_Connected_Component_After_Edge_Removal'] = 0
        results['Connected_Component_Size_Change_After_Edge_Removal'] = -original_largest_component_size
    
    # Add percentage changes
    results['Percentage_Of_Network_Intact_After_Node_Removal'] = (
        results['Largest_Connected_Component_After_Node_Removal'] / original_largest_component_size * 100
    )
    results['Percentage_Of_Network_Intact_After_Edge_Removal'] = (
        results['Largest_Connected_Component_After_Edge_Removal'] / original_largest_component_size * 100
    )
    
    return results

def identify_critical_paths(G: nx.Graph) -> Dict:
    """Identify critical paths between high-centrality nodes that could be attack vectors."""
    # Get top 10% of nodes by betweenness centrality
    betweenness = nx.betweenness_centrality(G)
    threshold = np.percentile(list(betweenness.values()), 90)
    critical_nodes = [n for n, c in betweenness.items() if c >= threshold]
    
    critical_paths = {}
    for source in critical_nodes:
        for target in critical_nodes:
            if source != target:
                try:
                    path = nx.shortest_path(G, source, target)
                    if len(path) > 2:  # Only paths with intermediate nodes
                        critical_paths[f"{source}->{target}"] = path
                except nx.NetworkXNoPath:
                    continue
    return critical_paths

def identify_bridge_nodes(G: nx.Graph) -> List:
    """Find nodes that, if removed, would significantly fragment the network."""
    bridges = []
    for node in G.nodes():
        G_temp = G.copy()
        G_temp.remove_node(node)
        original_components = nx.number_connected_components(G)
        new_components = nx.number_connected_components(G_temp)
        if new_components > original_components:
            bridges.append((node, new_components - original_components))
    return sorted(bridges, key=lambda x: x[1], reverse=True)

def calculate_network_segmentation(G: nx.Graph) -> Dict:
    """
    Analyze network segmentation to identify potential containment boundaries.
    
    Parameters:
    - G: NetworkX graph
    
    Returns:
    - Dictionary containing segmentation metrics
    """
    # Detect communities using Louvain method
    communities = community.best_partition(G)
    modularity = community.modularity(communities, G)
    
    # Count nodes in each segment
    segment_sizes = Counter(communities.values())
    
    # Count edges between segments
    cross_segment_edges = sum(1 for u, v in G.edges() 
                            if communities[u] != communities[v])
    
    # Calculate isolation score (ratio of internal to external edges)
    total_edges = G.number_of_edges()
    isolation_score = 1 - (cross_segment_edges / total_edges) if total_edges > 0 else 0
    
    segment_metrics = {
        'num_segments': len(set(communities.values())),
        'modularity': modularity,
        'segment_sizes': segment_sizes,
        'cross_segment_edges': cross_segment_edges,
        'isolation_score': isolation_score,
        'communities': communities  # Include community assignments for nodes
    }
    
    return segment_metrics

def find_potential_lateral_movement(G: nx.Graph) -> List[Tuple]:
    """Identify paths that could be used for lateral movement."""
    paths = []
    
    for component in nx.connected_components(G):
        subgraph = G.subgraph(component)
        if nx.is_connected(subgraph):
            # Find all pairs shortest paths
            all_pairs = dict(nx.all_pairs_shortest_path(subgraph))
            
            # Find the longest shortest path (diameter path)
            max_length = 0
            longest_path = None
            
            for source in all_pairs:
                for target, path in all_pairs[source].items():
                    if len(path) > max_length:
                        max_length = len(path)
                        longest_path = path
            
            if longest_path and len(longest_path) >= 3:  # Only consider paths that could represent lateral movement
                paths.append(longest_path)
    
    return paths

def generate_action_items(G1: nx.Graph, G2: nx.Graph, 
                         bridge_nodes: List, 
                         critical_paths: Dict,
                         segmentation: Dict,
                         anomalies: pd.DataFrame) -> pd.DataFrame:
    """Generate prioritized action items for network administrators."""
    action_items = []
    
    # Check for highly impactful bridge nodes
    for node, impact in bridge_nodes[:3]:  # Top 3 most critical
        action_items.append({
            'Priority': 'HIGH',
            'Category': 'Network Segmentation',
            'Finding': f'Critical node {node} could split network into {impact} segments if compromised',
            'Action': f'Implement redundant paths around node {node}; Consider network segmentation at this point'
        })
    
    # Analyze potential attack paths
    for path_name, path in list(critical_paths.items())[:3]:
        action_items.append({
            'Priority': 'HIGH',
            'Category': 'Attack Vector',
            'Finding': f'Critical path identified: {" -> ".join(map(str, path))}',
            'Action': 'Implement access controls and monitoring along this path'
        })
    
    # Check for anomalous nodes
    anomalous_nodes = anomalies[anomalies['anomaly'] == -1].index.tolist()
    for node in anomalous_nodes:
        action_items.append({
            'Priority': 'MEDIUM',
            'Category': 'Anomalous Behavior',
            'Finding': f'Node {node} shows unusual connectivity patterns',
            'Action': 'Investigate traffic patterns and implement enhanced monitoring'
        })
    
    # Assess segmentation issues
    if segmentation['cross_segment_edges'] > len(G2.edges()) * 0.3:  # More than 30% cross-segment
        action_items.append({
            'Priority': 'MEDIUM',
            'Category': 'Network Segmentation',
            'Finding': 'High number of cross-segment connections detected',
            'Action': 'Review and strengthen network segmentation policies'
        })
    
    return pd.DataFrame(action_items)

def visualize_network_overview(G1: nx.Graph, G2: nx.Graph, output_folder: str):
    """Create and display network visualization."""
    os.makedirs(output_folder, exist_ok=True)
    
    fig, axs = plt.subplots(2, 2, figsize=(20, 15))
    
    degrees1 = [d for n, d in G1.degree()]
    degrees2 = [d for n, d in G2.degree()]
    
    degrees1 = [np.nan if np.isinf(d) else d for d in degrees1]
    degrees2 = [np.nan if np.isinf(d) else d for d in degrees2]
    
    sns.histplot(degrees1, color='blue', label='Before', kde=False, stat="density", bins=20, alpha=0.5, ax=axs[0,0])
    sns.histplot(degrees2, color='green', label='After', kde=False, stat="density", bins=20, alpha=0.5, ax=axs[0,0])
    axs[0,0].set_title('Degree Distribution Comparison')
    axs[0,0].set_xlabel('Degree')
    axs[0,0].set_ylabel('Density')
    axs[0,0].legend()
    
    pos_before = nx.spring_layout(G1, seed=42)
    nx.draw(G1, pos_before, node_size=50, alpha=0.6, with_labels=False, node_color='blue', ax=axs[0,1])
    axs[0,1].set_title('Network Before Incident')
    
    pos_after = nx.spring_layout(G2, seed=42)
    nx.draw(G2, pos_after, node_size=50, alpha=0.6, with_labels=False, node_color='green', ax=axs[1,0])
    axs[1,0].set_title('Network After Incident')
    
    axs[1,1].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'network_overview.png'))
    plt.close()

def visualize_node_metrics_heatmap(centrality_changes: pd.DataFrame, output_folder: str):
    """Create and display heatmap of changes in node metrics."""
    plt.figure(figsize=(12, 16))
    
    sns.heatmap(centrality_changes, 
                annot=True,
                cmap='coolwarm',
                center=0,
                fmt='.3f',
                cbar_kws={'label': 'Change in Centrality'},
                square=False,
                linewidths=0.5)
    
    plt.title('Changes in Centrality Measures for All Nodes', pad=20)
    plt.xlabel('Centrality Metrics', labelpad=10)
    plt.ylabel('Nodes', labelpad=10)
    
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    
    plt.savefig(os.path.join(output_folder, 'node_metrics_changes_heatmap.png'))
    plt.close()

def visualize_critical_infrastructure(G: nx.Graph, 
                                    bridge_nodes: List,
                                    critical_paths: Dict,
                                    output_folder: str):
    """Create visualization highlighting critical infrastructure."""
    plt.figure(figsize=(15, 10))
    
    # Create layout
    pos = nx.spring_layout(G, k=1, iterations=50)
    
    # Draw base network
    nx.draw_networkx_edges(G, pos, alpha=0.2, edge_color='gray')
    
    # Draw nodes with size based on importance
    node_colors = []
    node_sizes = []
    bridge_nodes_set = {node for node, _ in bridge_nodes}
    
    for node in G.nodes():
        if node in bridge_nodes_set:
            node_colors.append('red')
            node_sizes.append(300)
        else:
            node_colors.append('lightblue')
            node_sizes.append(100)
    
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                          node_size=node_sizes)
    
    # Highlight critical paths
    for path in list(critical_paths.values())[:3]:  # Top 3 critical paths
        path_edges = list(zip(path[:-1], path[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, 
                             edge_color='red', width=2)
    
    # Add legend
    legend_elements = [
        mpatches.Patch(color='red', label='Bridge Nodes'),
        mpatches.Patch(color='lightblue', label='Regular Nodes'),
        mpatches.Patch(color='red', alpha=0.5, label='Critical Paths')
    ]
    plt.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.title("Critical Infrastructure Analysis")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'critical_infrastructure.png'))
    plt.close()

def display_summary_dashboard(action_items: pd.DataFrame, 
                            changes: Dict,
                            segmentation: Dict,
                            anomalies: pd.DataFrame):
    """Display a concise summary dashboard with key findings and metrics."""
    print("\n" + "="*50)
    print("INCIDENT RESPONSE SUMMARY DASHBOARD")
    print("="*50)
    
    print("\nPRIORITY ACTION ITEMS:")
    print(tabulate(action_items[action_items['Priority'] == 'HIGH'],
                  headers='keys', tablefmt='grid', showindex=False))
    
    print("\nKEY METRICS:")
    metrics_table = [
        ["New Connections", changes.get('New Edges', 0)],
        ["Removed Connections", changes.get('Removed Edges', 0)],
        ["Affected Nodes", changes.get('New Nodes', 0) + changes.get('Removed Nodes', 0)],
        ["Network Segments", segmentation['num_segments']],
        ["Anomalous Nodes", len(anomalies[anomalies['anomaly'] == -1])],
    ]
    print(tabulate(metrics_table, headers=['Metric', 'Value'], tablefmt='grid'))
    
    print("\nRECOMMENDED IMMEDIATE ACTIONS:")
    print("1. Review and action all HIGH priority items above")
    print("2. Monitor anomalous nodes for suspicious activity")
    print("3. Review new network connections for unauthorized changes")
    print("4. Implement segmentation at identified bridge nodes")
    print("\nDetailed analysis and visualizations saved to output folder.")

def save_metrics_to_csv(spectral_df: pd.DataFrame, other_changes_df: pd.DataFrame,
                       structural_changes_df: pd.DataFrame, output_folder: str):
    """Save metrics to CSV files."""
    spectral_df.to_csv(os.path.join(output_folder, 'spectral_metrics_changes.csv'), index=False)
    other_changes_df.to_csv(os.path.join(output_folder, 'other_network_changes.csv'), index=False)
    structural_changes_df.to_csv(os.path.join(output_folder, 'structural_changes.csv'), index=False)

def save_anomalies_to_csv(anomalies: pd.DataFrame, output_folder: str):
    """Save anomalies to CSV."""
    anomalies_to_save = anomalies[anomalies['anomaly'] == -1]
    anomalies_to_save.to_csv(os.path.join(output_folder, 'node_metrics_changes.csv'), index=False)

def save_action_items(action_items: pd.DataFrame, output_folder: str):
    """Save action items to both CSV and markdown formats."""
    # Save to CSV
    action_items.to_csv(os.path.join(output_folder, 'action_items.csv'), index=False)
    
    # Save to markdown
    with open(os.path.join(output_folder, 'action_items.md'), 'w') as f:
        f.write("# Incident Response Action Items\n\n")
        f.write("## Priority Actions\n\n")
        f.write(tabulate(action_items[action_items['Priority'] == 'HIGH'], 
                        headers='keys', tablefmt='pipe', showindex=False))
        f.write("\n\n## Secondary Actions\n\n")
        f.write(tabulate(action_items[action_items['Priority'] == 'MEDIUM'], 
                        headers='keys', tablefmt='pipe', showindex=False))

def save_additional_metrics(k_core2: Dict, assortativity1: float, assortativity2: float,
                          local_efficiency1: float, local_efficiency2: float,
                          robustness1: Dict, robustness2: Dict, output_folder: str):
    """Save additional metrics to text file."""
    with open(os.path.join(output_folder, 'additional_metrics.txt'), 'w') as f:
        f.write("=== K-Core Numbers (After) ===\n")
        for node, core in k_core2.items():
            f.write(f"{node}: {core}\n")
        
        f.write("\n=== Degree Assortativity Coefficient ===\n")
        f.write(f"Before: {assortativity1:.6f}, After: {assortativity2:.6f}\n")
        
        f.write("\n=== Local Efficiency ===\n")
        f.write(f"Before: {local_efficiency1:.6f}, After: {local_efficiency2:.6f}\n")
        
        f.write("\n=== Network Robustness ===\n")
        f.write("Before Removal:\n")
        for key, value in robustness1.items():
            f.write(f"  {key}: {value}\n")
        f.write("After Removal:\n")
        for key, value in robustness2.items():
            f.write(f"  {key}: {value}\n")

def create_readme(spectral_df: pd.DataFrame, other_changes_df: pd.DataFrame,
                 structural_changes_df: pd.DataFrame, anomalies: pd.DataFrame,
                 output_folder: str):
    """Create README.md summarizing metrics."""
    readme_path = os.path.join(output_folder, 'README.md')
    with open(readme_path, 'w') as f:
        f.write("# Post-Incident Network Analysis Report\n\n")
        
        f.write("## Network Overview\n\n")
        f.write("![Network Overview](network_overview.png)\n\n")
        
        f.write("## Critical Infrastructure Analysis\n\n")
        f.write("![Critical Infrastructure](critical_infrastructure.png)\n\n")
        
        f.write("## Spectral Metrics\n\n")
        f.write(spectral_df.to_markdown(index=False))
        f.write("\n\n")
        
        f.write("## Other Network-Level Changes\n\n")
        f.write(other_changes_df.to_markdown(index=False))
        f.write("\n\n")
        
        f.write("## Structural Changes\n\n")
        f.write(structural_changes_df.to_markdown(index=False))
        f.write("\n\n")
        
        f.write("## Node Metrics Changes Heatmap\n\n")
        f.write("![Node Metrics Changes Heatmap](node_metrics_changes_heatmap.png)\n\n")
        
        if not anomalies.empty:
            f.write("### Detected Anomalies\n\n")
            f.write(anomalies[anomalies['anomaly'] == -1].to_markdown(index=False))
        else:
            f.write("### No anomalies detected\n")
        f.write("\n\n")
        
        f.write("## Additional Metrics\n\n")
        with open(os.path.join(output_folder, 'additional_metrics.txt'), 'r') as metrics_file:
            f.write(f"```plaintext\n{metrics_file.read()}\n```\n")

def main():
    """Main execution function."""
    input_folder = r'C:\Users\Service Casket\Desktop\Network Analysis Tool\post-incident'
    nodes_file_before = os.path.join(input_folder, 'ICS_OT Nodes.csv')
    edges_file_before = os.path.join(input_folder, 'ICS_OT Edges.csv')
    nodes_file_after = os.path.join(input_folder, 'ICS_OT NodesInfected.csv')
    edges_file_after = os.path.join(input_folder, 'ICS_OT EdgesInfected.csv')
    output_folder = os.path.join(input_folder, 'output')
    os.makedirs(output_folder, exist_ok=True)

    # Load networks
    G1, nodes_df1, edges_df1 = load_graph(nodes_file_before, edges_file_before)
    G2, nodes_df2, edges_df2 = load_graph(nodes_file_after, edges_file_after)

    # Calculate metrics
    network_metrics1 = calculate_network_metrics(G1)
    network_metrics2 = calculate_network_metrics(G2)
    node_metrics1 = calculate_node_metrics(G1)
    node_metrics2 = calculate_node_metrics(G2)

    # Calculate changes
    changes = analyze_changes(G1, G2)
    
    # Calculate additional metrics
    k_core1 = calculate_k_core(G1)
    k_core2 = calculate_k_core(G2)
    edge_betweenness1 = calculate_edge_betweenness(G1)
    edge_betweenness2 = calculate_edge_betweenness(G2)
    node_strength1 = calculate_node_strength(G1)
    node_strength2 = calculate_node_strength(G2)
    assortativity1 = calculate_assortativity(G1)
    assortativity2 = calculate_assortativity(G2)
    local_efficiency1 = calculate_local_efficiency(G1)
    local_efficiency2 = calculate_local_efficiency(G2)
    robustness1 = assess_network_robustness(G1)
    robustness2 = assess_network_robustness(G2)

    # Additional analysis for incident response
    critical_paths = identify_critical_paths(G2)
    bridge_nodes = identify_bridge_nodes(G2)
    segmentation = calculate_network_segmentation(G2)
    lateral_paths = find_potential_lateral_movement(G2)

    # Calculate centrality changes
    centrality_metrics = ['Degree Centrality', 'Betweenness Centrality', 
                         'Closeness Centrality', 'Eigenvector Centrality']
    centrality_changes = node_metrics2[centrality_metrics] - node_metrics1[centrality_metrics]
    centrality_changes.rename(columns={
        'Degree Centrality': 'Degree',
        'Betweenness Centrality': 'Betweenness',
        'Closeness Centrality': 'Closeness',
        'Eigenvector Centrality': 'Eigenvector'
    }, inplace=True)

    # Detect anomalies
    anomalies = centrality_changes.copy()
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    anomalies['anomaly'] = iso_forest.fit_predict(centrality_changes.values)

    # Prepare DataFrames for output
    network_changes = {k: network_metrics2[k] - network_metrics1[k] 
                      for k in network_metrics1.keys()}
    spectral_metrics = ['Spectral Radius', 'Fiedler Eigenvalue']
    spectral_df = pd.DataFrame({
        'Metric': spectral_metrics,
        'Before': [network_metrics1.get(m) for m in spectral_metrics],
        'After': [network_metrics2.get(m) for m in spectral_metrics],
        'Change': [network_changes.get(m) for m in spectral_metrics]
    })

    other_changes_df = pd.DataFrame({
        'Metric': [k for k in network_changes.keys() if k not in spectral_metrics],
        'Change': [v for k, v in network_changes.items() if k not in spectral_metrics]
    })

    structural_changes_df = pd.DataFrame({
        'Structural Change': list(changes.keys()),
        'Value': list(changes.values())
    })

    # Generate action items
    action_items = generate_action_items(G1, G2, bridge_nodes, critical_paths,
                                       segmentation, anomalies)

    # Display visualizations and metrics
    print("\nGenerating incident response analysis...")
    visualize_network_overview(G1, G2, output_folder)
    visualize_node_metrics_heatmap(centrality_changes, output_folder)
    visualize_critical_infrastructure(G2, bridge_nodes, critical_paths, output_folder)
    
    # Display summary dashboard
    display_summary_dashboard(action_items, changes, segmentation, anomalies)
    
    # Save all results
    print("\nSaving detailed analysis to output folder...")
    save_metrics_to_csv(spectral_df, other_changes_df, structural_changes_df, output_folder)
    save_anomalies_to_csv(anomalies, output_folder)
    save_additional_metrics(k_core2, assortativity1, assortativity2,
                          local_efficiency1, local_efficiency2,
                          robustness1, robustness2, output_folder)
    save_action_items(action_items, output_folder)
    create_readme(spectral_df, other_changes_df, structural_changes_df, 
                 anomalies, output_folder)
    
    print(f"\nAnalysis complete. Results saved to: {output_folder}")
    print("\nRecommended next steps:")
    print("1. Review the action_items.md file for prioritized response actions")
    print("2. Examine critical_infrastructure.png to understand vulnerable network points")
    print("3. Check node_metrics_changes_heatmap.png for detailed node behavior changes")
    print("4. Review the README.md for a complete analysis summary")

if __name__ == "__main__":
    main()



Generating incident response analysis...

INCIDENT RESPONSE SUMMARY DASHBOARD

PRIORITY ACTION ITEMS:
+------------+----------------------+---------------------------------------------------------------------+---------------------------------------------------------------------------------------+
| Priority   | Category             | Finding                                                             | Action                                                                                |
+============+======================+=====================================================================+=======================================================================================+
| HIGH       | Network Segmentation | Critical node 23 could split network into 4 segments if compromised | Implement redundant paths around node 23; Consider network segmentation at this point |
+------------+----------------------+---------------------------------------------------------------------+------